In [36]:
import os
import glob
import numpy as np
import scipy.io
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import tensorflow_addons as tfa
import pandas as pd
import matplotlib.pyplot as plt
adam = tf.keras.optimizers.Adam(learning_rate=0.0005)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Input, Conv1D, MaxPooling1D, Flatten, Dense
import numpy as np
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Reshape

from keras.callbacks import ModelCheckpoint ,EarlyStopping
temperature = 0.03
learning_rate=0.001

In [37]:
def dataloader(path, featType):
    """
    Load data from a MATLAB file.

    Args:
        path (str): Path to the MATLAB file.
        featType (int): Type of features to load.

    Returns:
        Tuple: Tuple containing input features, labels, weights, and additional information.
    """
    data = scipy.io.loadmat(path)
    print(data.keys())

    AF = data['AF']
    x1 = AF[:-2]
    y = AF[-2]
    w = AF[-1]

    if featType == 1:
        x = x1
    else:
        x2 = data['CF']
        x = np.concatenate((x1, x2), axis=0)
    return x.T, y.T, w.T, data['CF_info']

def calculate_accuracy(arr1, arr2):
    """
    Calculate the accuracy between two arrays.

    Args:
        arr1 (array): First array.
        arr2 (array): Second array.

    Returns:
        float: Accuracy between the two arrays.
    """
    count = sum(1 for itr1, itr2 in zip(arr1, arr2) if itr1 == itr2)
    return count / len(arr1)

def normalization(feats):

    """
    Normalize the input features using standard scaling.

    Args:
        feats (array): Input features.

    Returns:
        array: Normalized features.
    """
    df = pd.DataFrame(feats)
    scaler = StandardScaler()
    x_new = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return x_new

def make_partitions(arr_words, arr_labels):

    """
    Create partitions based on word boundaries and labels.

    Args:
        arr_words (array): Array of words.
        arr_labels (array): Array of labels.

    Returns:
        array: Partitions based on word boundaries and labels.
    """
    v = []
    temp = []

    for i in range(len(arr_words) - 1):
        word = arr_words[i]
        next_word = arr_words[i + 1]
        temp.append(arr_labels[i])

        if word != next_word or i == len(arr_words) - 2:
            if i == len(arr_words) - 2:
                temp.append(arr_labels[i + 1])

            numpy_temp = np.array(temp)
            temp_max = np.amax(numpy_temp)
            numpy_temp = np.divide(numpy_temp, temp_max)
            v = np.concatenate((v, numpy_temp), axis=None)
            temp.clear()

    v1 = [1 if i == 1 else 0 for i in v]
    return v1

In [38]:
fatyp = 'TypicalFA_comb1'
drivepath = 'finalData/'+ fatyp +'/';
#featFiles = 'GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress'               #glob.glob(drivepath + '*train*')
filee = drivepath+'GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress.mat'
featType = 1; #Acoustic or Acoustic+context
if featType == 1:
  original_dim = 19
else:
  original_dim = 38

In [39]:
# print('Classification with::::::',os.path.basename(filee))

train_path = filee; test_path = filee.replace('train','test')
# print('test file:::::::',os.path.basename(test_path))
xtrain, ytrain, wtrain, info_train = dataloader(train_path, featType); 
xtrain1, ytrain1, wtrain1, info_train1 = dataloader(train_path, featType=2);
xtest, ytest, wtest ,info_test = dataloader(test_path, featType)
xtest1, ytest1, wtest1, info_test1 = dataloader(test_path, featType=2);



dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])


In [40]:
xtest_a = normalization(xtest)
xtest_ac = normalization(xtest1)
xtrain = normalization(xtrain)
xtrain1 = normalization(xtrain1)

woPP=[]; wPP=[]
input_shape1 = (19,1)
input_shape2 = (38,1)
temperature = 0.03
learning_rate=0.001

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# Splitting xtrain and ytrain into training and validation sets
xtra_a, xval_a, ytra_a, yval_a = train_test_split(xtrain, ytrain, test_size=0.2, random_state=42)

# Splitting xtrain1 and ytrain1 into training and validation sets
xtra_ac, xval_ac, ytra_ac, yval_ac = train_test_split(xtrain1, ytrain1, test_size=0.2, random_state=42)

In [78]:
def create_classifier(hidden_units):

    # for layer in encoder.layers:
    #     layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    # features = encoder(inputs)
    features=layers.Dense(hidden_units, activation="relu")(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    outputs = layers.Dense(1, activation="sigmoid")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[keras.metrics.BinaryAccuracy()],
    )
    return model

units=64

classifier = create_classifier(units)

# history = classifier.fit(x=xtra_ac, y=ytra_ac, validation_data =(xval_ac,yval_a), batch_size=batch_size, epochs=num_epochs, callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)])


In [80]:
classifier.summary()
classifier.fit(x=xtra_ac, y=ytra_ac, validation_data =(xval_ac,yval_a), batch_size=batch_size, epochs=num_epochs, callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)])

Model: "cifar10-classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_70 (InputLayer)       [(None, 38)]              0         
                                                                 
 dense_109 (Dense)           (None, 64)                2496      
                                                                 
 dropout_50 (Dropout)        (None, 64)                0         
                                                                 
 dense_110 (Dense)           (None, 32)                2080      
                                                                 
 dropout_51 (Dropout)        (None, 32)                0         
                                                                 
 dense_111 (Dense)           (None, 32)                1056      
                                                                 
 dense_112 (Dense)           (None, 32)         

/usr/local/lib/python3.8/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


295/295 [==============================] - 4s 5ms/step - loss: 0.5763 - binary_accuracy: 0.6933 - val_loss: 0.4817 - val_binary_accuracy: 0.7820
Epoch 2/200
  1/295 [..............................] - ETA: 1s - loss: 0.5974 - binary_accuracy: 0.7500

/usr/local/lib/python3.8/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


295/295 [==============================] - 1s 5ms/step - loss: 0.4898 - binary_accuracy: 0.7716 - val_loss: 0.4311 - val_binary_accuracy: 0.8134
Epoch 3/200
295/295 [==============================] - 1s 4ms/step - loss: 0.4473 - binary_accuracy: 0.7950 - val_loss: 0.4103 - val_binary_accuracy: 0.8202
Epoch 4/200
295/295 [==============================] - 1s 4ms/step - loss: 0.4371 - binary_accuracy: 0.8094 - val_loss: 0.4003 - val_binary_accuracy: 0.8287
Epoch 5/200
295/295 [==============================] - 1s 5ms/step - loss: 0.4129 - binary_accuracy: 0.8145 - val_loss: 0.3727 - val_binary_accuracy: 0.8422
Epoch 6/200
295/295 [==============================] - 1s 4ms/step - loss: 0.3933 - binary_accuracy: 0.8283 - val_loss: 0.3662 - val_binary_accuracy: 0.8431
Epoch 7/200
295/295 [==============================] - 1s 5ms/step - loss: 0.3897 - binary_accuracy: 0.8262 - val_loss: 0.3499 - val_binary_accuracy: 0.8541
Epoch 8/200
295/295 [==============================] - 1s 5ms/step - l

In [90]:
from sklearn.metrics import accuracy_score

150/150 [==============================] - 0s 1ms/step


In [97]:
y_pred = classifier.predict(xtest_ac)
y_pred = np.round(y_pred).flatten()
accuracy = accuracy_score(ytest, y_pred)

from sklearn.metrics import f1_score

f1 = f1_score(ytest, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)

150/150 [==============================] - 0s 1ms/step
Accuracy: 0.8993526832324076
F1 Score: 0.8856193640246796
